# Macro-Financial Linkages: A Quant-Research Perspective
## Engineering Systematic Macro Features for Regime-Switching Models (2000–2025)

---

### Research Objective
Traditional macroeconomic analysis often focuses on point-in-time event studies or static levels. From a quantitative finance perspective, however, the value of macroeconomic data lies in its **stochastic innovations (shocks)** and its role as a **regime filter** for asset returns. 

This research workflow transforms raw macroeconomic indicators into a professional feature set. We hypothesize that financial markets do not respond linearly to macro levels but rather systematically encode the **departure from trend** and the **interaction between states** (e.g., inflationary growth vs. deflationary recession).

### Methodology Overview
1. **Information Ingestion**: Centralizing disparate frequency data (monthly, quarterly) into a daily-aligned temporal framework.
2. **The Macro State Layer**: Mapping slow-moving growth and inflation dynamics into a four-quadrant regime model.
3. **Standardized Shocks**: Normalizing macro innovations by realized volatility to capture "surprise" magnitude.
4. **Cross-Asset Sensitivity**: Projecting macro states onto equity and commodity risk premia.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings

# Professional Plotting Configuration
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['axes.titlesize'] = 14
plt.style.use('ggplot')
sns.set_theme(style="whitegrid", palette="viridis")
warnings.filterwarnings("ignore")

print("Environment initialized: High-fidelity research mode active.")

## Section 1: Data Ingestion & Alignment

Quantitative macro research requires a unified timeline. We integrate the preprocessed macro indicators with high-frequency asset prices, ensuring all information is correctly lagged to avoid look-ahead bias.

In [ ]:
def load_macro_quant_data(base_dir=".."):
    """
    Loads macro and asset datasets, ensuring they are aligned on a daily business day index.
    """
    # Load Macro
    macro_path = os.path.join(base_dir, "data/processed/macro_processed.csv")
    if not os.path.exists(macro_path):
        macro_path = "data/processed/macro_processed.csv"
    macro = pd.read_csv(macro_path, index_col=0, parse_dates=True)
    
    # Load Core Assets (SPY as Proxy for Equities, GLD for Commodities)
    # We iterate through the raw directories to build a multi-asset panel
    assets = {}
    for folder in ["equities", "commodities", "sectors"]:
        dir_path = os.path.join(base_dir, "data/raw", folder)
        if not os.path.exists(dir_path):
            dir_path = os.path.join("data/raw", folder)
            
        for f in [x for x in os.listdir(dir_path) if x.endswith(".csv")]:
            ticker = f.replace(".csv", "")
            df = pd.read_csv(os.path.join(dir_path, f), index_col=0, parse_dates=True)
            assets[ticker] = df.iloc[:, 0] # Assume first column is the target price
            
    prices = pd.DataFrame(assets).sort_index()
    
    # Align
    common_idx = macro.index.intersection(prices.index)
    return macro.loc[common_idx], prices.loc[common_idx]

macro_raw, prices_raw = load_macro_quant_data()
print(f"Ingested {macro_raw.shape[1]} macro series and {prices_raw.shape[1]} asset tickers.")
print(f"Time-span: {macro_raw.index.min().date()} to {macro_raw.index.max().date()}")

## Section 2: The Macro State Layer (State Filter)

### Economic Intuition
Market participants do not price information in a vacuum. The propagation of a shock is conditional on the **Macro Regime**. We apply a Z-score transformation to define "Extreme" vs "Normal" states and construct a **Growth-Inflation Quadrant**.

**Regime Definition:**
- **Goldilocks**: High Growth, Low Inflation
- **Stagflation**: Low Growth, High Inflation
- **Reflation**: High Growth, High Inflation
- **Deflationary Recession**: Low Growth, Low Inflation

In [ ]:
def engineer_state_features(df):
    """
    Transforms raw levels into standardized state indicators.
    """
    states = pd.DataFrame(index=df.index)
    
    # 1. Growth Proxy (Industrial Production YoY)
    states['growth_z'] = (df['INDPRO'].pct_change(252) - df['INDPRO'].pct_change(252).rolling(1260).mean()) / df['INDPRO'].pct_change(252).rolling(1260).std()
    
    # 2. Inflation Proxy (CPI YoY)
    states['inflation_z'] = (df['CPIAUCSL'].pct_change(252) - df['CPIAUCSL'].pct_change(252).rolling(1260).mean()) / df['CPIAUCSL'].pct_change(252).rolling(1260).std()
    
    # 3. Monetary Tightness (Fed Funds vs 10Y Yield)
    states['yield_curve'] = df['DGS10'] - df['FEDFUNDS']
    
    # 4. Sentiment (Michigan Consumer Sentiment standardized)
    states['sentiment_z'] = (df['UMCSENT'] - df['UMCSENT'].rolling(1260).mean()) / df['UMCSENT'].rolling(1260).std()
    
    # Define Quadrants (Binary Filters)
    states['is_high_growth'] = (states['growth_z'] > 0).astype(int)
    states['is_high_inflation'] = (states['inflation_z'] > 0).astype(int)
    
    return states.dropna()

macro_states = engineer_state_features(macro_raw)

fig, ax = plt.subplots(1, 2, figsize=(16, 5))
sns.lineplot(data=macro_states[['growth_z', 'inflation_z']], ax=ax[0])
ax[0].set_title("Standardized Growth & Inflation States")
ax[0].axhline(0, color='black', linestyle='--')

macro_states[['is_high_growth', 'is_high_inflation']].value_counts().plot(kind='bar', ax=ax[1])
ax[1].set_title("Frequency of Macro Quadrants")
plt.show()

## Section 3: Shocks & Information Catalysts

### Statistical Rationale
To build predictive signals, we must convert non-stationary macro levels into **stationary innovations**. We define a "Macro Shock" as the daily innovation normalized by its rolling standard deviation. This allows us to compare the relative impact of a VIX spike vs a Fed Funds move on the same scale.

In [ ]:
def engineer_shock_features(df):
    """
    Constructs volatility-normalized shocks for market-moving variables.
    """
    shocks = pd.DataFrame(index=df.index)
    
    # Interest Rate Shocks
    shocks['rate_shock'] = df['DGS10'].diff() / df['DGS10'].diff().rolling(60).std()
    
    # Energy Shocks (Log returns normalized)
    shocks['energy_shock'] = np.log(df['DCOILWTICO']).diff() / np.log(df['DCOILWTICO']).diff().rolling(60).std()
    
    # Volatility Shocks (VIX levels change normalized)
    shocks['vol_shock'] = df['VIXCLS'].diff() / df['VIXCLS'].diff().rolling(60).std()
    
    return shocks.dropna()

macro_shocks = engineer_shock_features(macro_raw)
sns.histplot(macro_shocks['rate_shock'], kde=True, color='teal')
plt.title("Distribution of Normalized Interest Rate Shocks")
plt.show()

## Section 2: Macro-Financial Sensitivity (Dynamic Linkages)

### The Research Question
How does the linkage between equities (SPY) and commodities (GLD) shift across macro states? We use rolling correlations weighted by macro shocks to identify periods of **regime decoupling**.

In [ ]:
# Calculate returns
returns = np.log(prices_raw).diff().dropna()

# Rolling Correlation: Equities vs Commodities
rolling_corr = returns['SPY'].rolling(126).corr(returns['GLD'])

plt.figure(figsize=(14, 6))
plt.plot(rolling_corr, label='SPY-GLD 6M Rolling Correlation', color='darkblue')
plt.fill_between(rolling_corr.index, 0, 1, where=(macro_states['is_high_inflation'] == 1), 
                 color='red', alpha=0.1, label='High Inflation Regime')
plt.title("Asset Correlation Dynamics across Inflationary Regimes")
plt.legend()
plt.show()

## Section 5: Integrated Feature Export

Finally, we consolidate our engineered states and shocks into a single master feature file for subsequent Markov-Switching modeling.

In [ ]:
final_features = pd.concat([macro_states, macro_shocks], axis=1).dropna()
output_path = "../data/processed/integrated_features.csv"
if not os.path.exists("../data/processed"):
    output_path = "data/processed/integrated_features.csv"

final_features.to_csv(output_path)
print(f"Success: {final_features.shape[0]} samples exported to {output_path}")
final_features.tail()